In [6]:
import numpy as np
execfile('midi_to_state.py')
daft_state = midiToNoteStateMatrix('daft_punk-one_more_time.mid')
len(daft_state)

2401

In [2]:
def encode_midi_states(state_mat):
    chord_to_idx = {}
    idx_to_chord = {}
    for sixteenth_note in state_mat:
        notes_only = [note[0] for note in sixteenth_note]
        note_idcs_set = tuple([i for i, x in enumerate(notes_only) if x == 1])
        if len(note_idcs_set) == 0: note_idcs_set = (None)
        if not note_idcs_set in chord_to_idx.keys():
            new_idx = len(chord_to_idx) + 1
            chord_to_idx[note_idcs_set] = new_idx
            idx_to_chord[new_idx] = note_idcs_set
        else:
            note_set_idx = chord_to_idx[note_idcs_set]
    return [chord_to_idx, idx_to_chord]

results = encode_midi_states(daft_state)
print (len(results[0]))
print (len(results[1]))
chord_to_idx = results[0]
idx_to_chord = results[1]

381
381


In [3]:
import random
randnum = random.sample(range(len(results[0])),1)[0]
print randnum
assert(results[0][results[1][randnum]] == randnum)

301


In [5]:
# now the set of chords is a library of possible classes
# return a num_notes x len_chord_set
def encode_song(midi_states, chord_to_idx):
    song_encoded = np.zeros((len(midi_states)+1, len(chord_to_idx)+1))
    for index in range(len(midi_states)):
        notes_only = [note[0] for note in midi_states[index]]
        note_idcs_set = tuple([i for i, x in enumerate(notes_only) if x == 1])
        if len(note_idcs_set) == 0: note_idcs_set = (None)
        library_idx = chord_to_idx[note_idcs_set]
        song_encoded.itemset((index,library_idx), 1)
    return song_encoded

encoded_song = encode_song(daft_state, results[0])
len(encoded_song)

2402

In [7]:
randnum = random.sample(range(len(encoded_song)),1)[0]
print randnum
assert(sum(encoded_song[randnum]) == 1)

2241


In [20]:
chord_to_idx = results[0]
idx_to_chord = results[1]

maxlen = 40
step = 3
chunks = []
next_chords = []
for i in range(0, len(encoded_song) - maxlen, step):
    chunks.append(encoded_song[i: i + maxlen])
    next_chords.append(encoded_song[i + maxlen])

# len(chunks) should == len(daft_state)/step
assert(len(chunks) == len(next_chords))

X = np.array(chunks)
y = np.array(next_chords)
print(X.shape)
print(y.shape)
print(sum(y[0]))

(788, 40, 382)
(788, 382)
1.0


In [31]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM

# build the model: 2 stacked LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(maxlen, len(chord_to_idx)+1)))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(len(chord_to_idx)+1))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

Build model...


In [33]:
model.fit(X, y, batch_size=128, nb_epoch=10)

Epoch 1/10
788/788 [==============================] - 2s - loss: 5.0600 - acc: 0.0660     
Epoch 2/10
788/788 [==============================] - 2s - loss: 4.6953 - acc: 0.1091     
Epoch 3/10
788/788 [==============================] - 2s - loss: 4.7714 - acc: 0.0774     
Epoch 4/10
788/788 [==============================] - 2s - loss: 4.5048 - acc: 0.1256     
Epoch 5/10
788/788 [==============================] - 2s - loss: 4.6337 - acc: 0.1168     
Epoch 6/10
788/788 [==============================] - 2s - loss: 4.4858 - acc: 0.1193     
Epoch 7/10
788/788 [==============================] - 2s - loss: 4.4890 - acc: 0.1053     
Epoch 8/10
788/788 [==============================] - 2s - loss: 4.2899 - acc: 0.1320     
Epoch 9/10
788/788 [==============================] - 2s - loss: 4.1255 - acc: 0.1320     
Epoch 10/10
788/788 [==============================] - 2s - loss: 4.1638 - acc: 0.1409     
